In [17]:
import os
import numpy as np
import pandas as pd
import cv2
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle

In [2]:
# Define directory paths
cat_dir_path = "C:/Users/Admin/Desktop/internship_data/SVM_for_cat_dog_classification/data_img/Cat"
dog_dir_path = "C:/Users/Admin/Desktop/internship_data/SVM_for_cat_dog_classification/data_img/Dog"
test_dir_path = "C:/Users/Admin/Desktop/internship_data/SVM_for_cat_dog_classification/data_img/test1"

# Define categories
categories = ['Cat', 'Dog']

In [3]:
# Function to extract HOG features from an image
def extract_hog_features(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(image, (128, 128))  # Resize image to 128x128
    features, _ = hog(resized_image, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)
    return features

In [4]:
# Load dataset and extract features
def load_dataset_and_extract_features(cat_path, dog_path):
    features = []
    labels = []
    
    for img_name in os.listdir(cat_path):
        img_path = os.path.join(cat_path, img_name)
        try:
            hog_features = extract_hog_features(img_path)
            features.append(hog_features)
            labels.append(0)  # Label for Cat
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            pass
    
    for img_name in os.listdir(dog_path):
        img_path = os.path.join(dog_path, img_name)
        try:
            hog_features = extract_hog_features(img_path)
            features.append(hog_features)
            labels.append(1)  # Label for Dog
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            pass
    
    return np.array(features), np.array(labels)

In [5]:
# Extract features from training data
X, y = load_dataset_and_extract_features(cat_dir_path, dog_dir_path)

In [6]:
# Ensure the features and labels are correctly shaped
print(f'Features shape: {X.shape}')  # Should be (number of samples, number of features)
print(f'Labels shape: {y.shape}')    # Should be (number of samples,)

Features shape: (25000, 8100)
Labels shape: (25000,)


In [7]:
# Split data into training and validation sets with an 80-20 split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
for i in range(0,10):
    print(X_train)

[[0.24605061 0.00762271 0.02257207 ... 0.25254032 0.16362757 0.09569485]
 [0.22287905 0.1014943  0.0832391  ... 0.14454524 0.06969969 0.09649322]
 [0.30304632 0.11047478 0.10030135 ... 0.02974479 0.12736642 0.30872343]
 ...
 [0.22662441 0.17015126 0.13522236 ... 0.15936413 0.13627661 0.03799306]
 [0.17712887 0.08781009 0.13556126 ... 0.18726031 0.11318644 0.10287446]
 [0.08817383 0.00535315 0.00118231 ... 0.38230453 0.         0.02018725]]
[[0.24605061 0.00762271 0.02257207 ... 0.25254032 0.16362757 0.09569485]
 [0.22287905 0.1014943  0.0832391  ... 0.14454524 0.06969969 0.09649322]
 [0.30304632 0.11047478 0.10030135 ... 0.02974479 0.12736642 0.30872343]
 ...
 [0.22662441 0.17015126 0.13522236 ... 0.15936413 0.13627661 0.03799306]
 [0.17712887 0.08781009 0.13556126 ... 0.18726031 0.11318644 0.10287446]
 [0.08817383 0.00535315 0.00118231 ... 0.38230453 0.         0.02018725]]
[[0.24605061 0.00762271 0.02257207 ... 0.25254032 0.16362757 0.09569485]
 [0.22287905 0.1014943  0.0832391  ... 

In [9]:
# Define the SVM model with fixed parameters
svm = SVC(C=1, kernel='rbf', gamma='auto')

# Train the model
svm.fit(X_train, y_train)

SVC(C=1, gamma='auto')

In [11]:
# Predict on the validation data
y_pred = svm.predict(X_val)

In [12]:
# Evaluate the accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


Validation Accuracy: 71.06%


In [13]:
# Save the trained model
with open('svm_hog_model.sav', 'wb') as model_file:
    pickle.dump(svm, model_file)

# Load the saved model
with open('svm_hog_model.sav', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

In [14]:
# Function to predict on new images
def predict_new_images(directory, model):
    new_data = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        try:
            hog_features = extract_hog_features(img_path)
            new_data.append(hog_features)
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            pass
    new_data = np.array(new_data)
    predictions = model.predict(new_data)
    return predictions

In [15]:
# Predict on new images
new_predictions = predict_new_images(test_dir_path, loaded_model)

In [16]:
# Print predictions for the new images
for i, prediction in enumerate(new_predictions):
    print(f'Image {i+1}: {categories[prediction]}')

Image 1: Dog
Image 2: Dog
Image 3: Dog
Image 4: Dog
Image 5: Dog
Image 6: Cat
Image 7: Cat
Image 8: Dog
Image 9: Cat
Image 10: Cat
Image 11: Cat
Image 12: Cat
Image 13: Cat
Image 14: Cat
Image 15: Cat
Image 16: Dog
Image 17: Dog
Image 18: Cat
Image 19: Dog
Image 20: Cat
Image 21: Dog
Image 22: Cat
Image 23: Cat
Image 24: Cat
Image 25: Cat
Image 26: Dog
Image 27: Cat
Image 28: Cat
Image 29: Dog
Image 30: Dog
Image 31: Dog
Image 32: Dog
Image 33: Dog
Image 34: Dog
Image 35: Cat
Image 36: Dog
Image 37: Dog
Image 38: Dog
Image 39: Dog
Image 40: Cat
Image 41: Cat
Image 42: Dog
Image 43: Cat
Image 44: Dog
Image 45: Cat
Image 46: Cat
Image 47: Dog
Image 48: Dog
Image 49: Dog
Image 50: Dog
Image 51: Dog
Image 52: Dog
Image 53: Cat
Image 54: Dog
Image 55: Cat
Image 56: Dog
Image 57: Dog
Image 58: Cat
Image 59: Cat
Image 60: Cat
Image 61: Dog
Image 62: Cat
Image 63: Dog
Image 64: Cat
Image 65: Dog
Image 66: Dog
Image 67: Cat
Image 68: Dog
Image 69: Dog
Image 70: Dog
Image 71: Dog
Image 72: Dog
I